##### Copyright 2025 Google LLC.

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Similarity Search using Qdrant

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/qdrant/Qdrant_similarity_search.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

## Overview

The [Gemini API](https://ai.google.dev/models/gemini) provides access to a family of generative AI models for generating content and solving problems. These models are designed and trained to handle both text and images as input.

[Qdrant](https://qdrant.tech/) is an open-source vector similarity search engine designed for efficient and scalable semantic search. It offers a simple yet powerful API to store and search high-dimensional vectors, supports filtering with metadata (payloads), and integrates easily into production systems. Qdrant can be self-hosted or accessed via its managed cloud service, making it quick to set up and ideal for a wide range of AI applications that rely on semantic understanding and retrieval.

In this notebook, you'll learn how to perform a similarity search on data from a website with the help of Gemini API and Qdrant.


## Setup

First, you must install the packages and set the necessary environment variables.

### Installation

Install google's python client SDK for the Gemini API, `google-genai`. Next, install Qdrant's Python client SDK, `qdrant-client`.

In [2]:
%pip install -q "google-genai>=1.0.0"
%pip install -q protobuf==4.25.1 qdrant-client[fastembed]

## Configure your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [3]:
from google.colab import userdata
from google import genai

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai_client = genai.Client(api_key=GOOGLE_API_KEY)

## Building the Movie Vector Index
This section covers preparing the movie dataset, generating embeddings using Gemini, and indexing them in Qdrant for similarity search.

### 1. Load the Dataset from Kaggle

Begin by loading the dataset from Kaggle using the kagglehub library. The dataset used in this notebook is the TMDB Movie Dataset 2023, which contains approximately 1 Million+ movie entries.

In [4]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "TMDB_movie_dataset_v11.csv"

df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "asaniczka/tmdb-movies-dataset-2023-930k-movies",
  file_path,
)


<ipython-input-4-be1b43b299ac>:6: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


### 2. Inspect the Dataset Structure

Perform a basic inspection to understand the data schema and quality of this dataset.

In [5]:
print("\nDataset Columns:")
print(df.columns)

print("\nMissing Values per Column:")
print(df.isnull().sum())

print(f"\nNumber of rows: {len(df)}")
print(f"Number of unique IDs: {df['id'].nunique()}")


Dataset Columns:
Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'adult', 'backdrop_path', 'budget', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'keywords'],
      dtype='object')

Missing Values per Column:
id                            0
title                        13
vote_average                  0
vote_count                    0
status                        0
release_date             230430
revenue                       0
runtime                       0
adult                         0
backdrop_path            915284
budget                        0
homepage                1105934
imdb_id                  609631
original_language             0
original_title               13
overview                 263410
popularity                    0
poster_path 

### 3. Filter and Clean the Dataset

This step filters the dataset to include only the relevant columns (`id`, `title`, `overview`, `genres`, `keywords`, `tagline`, and `release_date`) and removes entries missing critical information such as the movie title or both `overview` and `genres`.


In [6]:
import pandas as pd
import numpy as np
import ast
print(f"Original rows: {len(df)}")

columns_to_keep = ['id', 'title', 'overview', 'genres', 'keywords', 'tagline', 'release_date']

df_relevant = df[columns_to_keep].copy()

print(f"Rows before dropping missing title: {len(df_relevant)}")
df_relevant.dropna(subset=['title'], inplace=True)
df_relevant = df_relevant[~(df_relevant['genres'].isna() & df_relevant['overview'].isna())]
print(f"Rows after dropping missing title and dropping missing (genres and overview): {len(df_relevant)}")

# Fill missing text columns with empty strings
text_cols_to_fill = ['overview', 'genres', 'keywords', 'tagline']
for col in text_cols_to_fill:
    df_relevant[col] = df_relevant[col].fillna('')


# Extract release year from the release_date string
def get_year(date_str):
    if pd.isna(date_str) or not isinstance(date_str, str) or len(date_str) < 4:
        return None
    try:
        return int(date_str[:4])
    except (ValueError, TypeError):
        return None

df_relevant['release_year'] = df_relevant['release_date'].apply(get_year)

print("\nSample data after cleaning (keeping missing overviews):")
print(df_relevant[['id', 'title', 'overview', 'genres', 'keywords', 'tagline', 'release_year']].head())

Original rows: 1235247
Rows before dropping missing title: 1235247
Rows after dropping missing title and dropping missing (genres and overview): 1095533

Sample data after cleaning (keeping missing overviews):
       id            title                                           overview  \
0   27205        Inception  Cobb, a skilled thief who commits corporate es...   
1  157336     Interstellar  The adventures of a group of explorers who mak...   
2     155  The Dark Knight  Batman raises the stakes in his war on crime. ...   
3   19995           Avatar  In the 22nd century, a paraplegic Marine is di...   
4   24428     The Avengers  When an unexpected enemy emerges and threatens...   

                                        genres  \
0           Action, Science Fiction, Adventure   
1            Adventure, Drama, Science Fiction   
2               Drama, Action, Crime, Thriller   
3  Action, Adventure, Fantasy, Science Fiction   
4           Science Fiction, Action, Adventure   

  

### 4. Prepare Text for Embedding

This step prepares the movie metadata for embedding by combining relevant fields into a single structured text string. This representation includes the title, overview, genres, keywords, tagline, and release year (if available). The output is stored in a new column called `text_for_embedding`.

Embeddings are numerical vector representations of text that capture semantic meaning and relationships. These vectors can be used for tasks like similarity search and clustering.
Learn more about [text embeddings](https://ai.google.dev/gemini-api/docs/embeddings) and explore the [Gemini embedding notebook](https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/embedding_text_with_gemini.ipynb).

In [7]:
def create_embedding_text(row):
    """Combines available movie metadata into a single string for embedding."""
    # Title is always present, so it can be included directly
    title_str = f"Title: {row['title']}"
    overview_str = f"Overview: {row['overview']}" if row['overview'] else ""
    year_str = f"Release Year: {int(row['release_year'])}" if pd.notna(row['release_year']) else ""
    genre_str = f"Genres: {row['genres']}" if row['genres'] else ""
    keywords_str = f"Keywords: {row['keywords']}" if row['keywords'] else ""
    tagline_str = f"Tagline: {row['tagline']}" if row['tagline'] else ""

    parts = [
        title_str,
        overview_str,
        year_str,
        genre_str,
        keywords_str,
        tagline_str
    ]
    return "\n".join(part for part in parts if part)

df_relevant['text_for_embedding'] = df_relevant.apply(create_embedding_text, axis=1)

# Use this to inspect how movie data has been transformed for embedding
print(df_relevant[['id', 'title', 'text_for_embedding']].head())

       id            title                                 text_for_embedding
0   27205        Inception  Title: Inception\nOverview: Cobb, a skilled th...
1  157336     Interstellar  Title: Interstellar\nOverview: The adventures ...
2     155  The Dark Knight  Title: The Dark Knight\nOverview: Batman raise...
3   19995           Avatar  Title: Avatar\nOverview: In the 22nd century, ...
4   24428     The Avengers  Title: The Avengers\nOverview: When an unexpec...


### 5. Sample a Subset for Development

To improve efficiency during development, especially when working with resource-intensive processes like embedding generation and vector indexing, it is common to work with a smaller subset of the full dataset. This step randomly samples 5,000 movies from the cleaned dataset if it contains more entries. If the dataset is already smaller than the sample size, the entire dataset is used.

In [8]:
SAMPLE_SIZE = 5000

if len(df_relevant) > SAMPLE_SIZE:
    print(f"\nTaking a random sample of {SAMPLE_SIZE} movies for development.")
    df_sample = df_relevant.sample(n=SAMPLE_SIZE, random_state=42)
else:
    print(f"\nCleaned dataset size ({len(df_relevant)}) is smaller than or equal to SAMPLE_SIZE. Using the full cleaned dataset.")
    df_sample = df_relevant

print(f"Working with {len(df_sample)} movies for the next steps.")
print(df_sample[['id', 'title', 'release_year']].head())

columns_for_payload = ['title', 'overview', 'genres', 'keywords', 'tagline', 'release_year']
columns_final = ['id', 'text_for_embedding'] + columns_for_payload
df_sample = df_sample[columns_final]

print("\nFinal sample DataFrame structure for embedding/indexing:")
print(df_sample.info())



Taking a random sample of 5000 movies for development.
Working with 5000 movies for the next steps.
              id                       title  release_year
89006     787356                           X        2021.0
1085978  1005027  Pistruiatul 2: Ascunzișuri           NaN
1105919   938917                  Chanakyudu        2012.0
617410   1491546               She's Legal 3        2018.0
438702   1086816             Pleasure Vision        2017.0

Final sample DataFrame structure for embedding/indexing:
<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 89006 to 1158379
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  5000 non-null   int64  
 1   text_for_embedding  5000 non-null   object 
 2   title               5000 non-null   object 
 3   overview            5000 non-null   object 
 4   genres              5000 non-null   object 
 5   keywords            5000 non-nu

### 6. Initialize Qdrant for Vector Indexing



With the data prepared, the next step is to set up **Qdrant**, a vector similarity search engine optimized for storing and querying high-dimensional vectors. It supports fast indexing, filtering, and similarity search across millions of vectors.

Qdrant can run:

* Locally as a standalone service
* In the cloud for production deployments
* Or entirely **in-memory** for fast, temporary use during development

In this notebook, Qdrant is initialized using in-memory mode by passing `":memory:"` to the client. This stores data only in RAM, meaning it **will not persist after the session ends**. This is suitable for experimentation but not for saving results long-term.

You also configure the following:

* `COLLECTION_NAME`: The name of the Qdrant collection to store movie vectors
* `VECTOR_SIZE`: Set to `768` to match the dimensionality of the text embeddings generated by Gemini
* `DISTANCE_METRIC`: Set to **cosine distance**, which is ideal for measuring semantic similarity between embedding vectors

In [9]:
from qdrant_client import QdrantClient, models
import time

COLLECTION_NAME = "tmdb_movies_sample"

VECTOR_SIZE = 768
DISTANCE_METRIC = models.Distance.COSINE


# Initialize Qdrant client using in-memory storage
client = QdrantClient(":memory:")

### 7. Define Batch Embedding Function

This step defines a function to generate embeddings for movie text data in batches using the Gemini embedding model, with automatic retries for robustness.

In [10]:
import time
from google.api_core import exceptions, retry

MODEL_FOR_EMBEDDING = "embedding-001" # @param ["embedding-001", "text-embedding-004","gemini-embedding-exp-03-07"] {"allow-input":true, isTemplate: true}

BATCH_SIZE = 100
QDRANT_BATCH_SIZE = 768


@retry.Retry(timeout=3000)
def get_embeddings_batch(texts: list[str], task_type="RETRIEVAL_DOCUMENT") -> list[list[float]] | None:
    """
    Generates embeddings for a batch of texts using Gemini API with retry.

    Args:
        texts: A list of strings to embed.
        task_type: The task type for the embedding model.

    Returns:
        A list of embedding vectors (list of floats), or None if a non-retryable error occurs.
    """
    if not texts:
        return []
    try:
        response = genai_client.models.embed_content(
          model=MODEL_FOR_EMBEDDING,
          contents=texts,
          config={
            "task_type":task_type,
          }
        )
        return response.embeddings
    except exceptions.RetryError as e:
        print(f"Embedding batch failed after retries: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during embedding: {e}")
        return None

print(f"Batch embedding function 'get_embeddings_batch' defined using model: {MODEL_FOR_EMBEDDING}")

Batch embedding function 'get_embeddings_batch' defined using model: embedding-001


### 8. Create a Collection in Qdrant


A collection in Qdrant is like a table in a database, it stores vectors along with optional metadata (payload). Each collection has its own configuration, including vector size and similarity metric.

In [11]:
# In case someone tries running the whole notebook again they would want to create the collection again

try:
    client.delete_collection(collection_name=COLLECTION_NAME)
    print(f"Existing collection '{COLLECTION_NAME}' deleted.")
except Exception as e:
    print(f"Error deleting collection (it might not exist): {e}")

try:
    client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=models.VectorParams(
            size=VECTOR_SIZE,
            distance=DISTANCE_METRIC
        )
    )
    print(f"Collection '{COLLECTION_NAME}' created successfully.")
except Exception as e:
    print(f"Error creating collection: {e}")

Existing collection 'tmdb_movies_sample' deleted.
Collection 'tmdb_movies_sample' created successfully.


### 9. Create Payloads for Metadata Storage


In Qdrant, besides storing vector embeddings, you can attach additional information called payload to each vector. This metadata helps in filtering or retrieving relevant results based on attributes like title, genres, or release year.

The function below prepares the payload by extracting specified columns from each movie record, handling missing values and ensuring data types are compatible with Qdrant.

In [12]:
def create_payload(row, payload_columns):
    payload = {}
    for col in payload_columns:
        value = row[col]
        if pd.isna(value):
            payload[col] = None
        elif isinstance(value, (np.int64, np.int32)):
            payload[col] = int(value)
        elif isinstance(value, (np.float64, np.float32)):
             payload[col] = float(value)
        else:
            payload[col] = value
    return payload

payload_columns = [
    'title', 'overview', 'genres', 'keywords', 'tagline', 'release_year'
]

### 10. Batch Embedding and Indexing to Qdrant



This step processes the sampled movies dataset in batches to generate vector embeddings using the Gemini API and upload (upsert) these embeddings along with their metadata payloads to the Qdrant collection.

**Key points of this process:**

* The dataset is divided into batches of size `BATCH_SIZE` for embedding generation to stay within API limits.
* Each batch's text data is sent to the Gemini embedding API with retries handled in the embedding function.
* For every successfully embedded batch, the code prepares **points** (each containing an ID, vector embedding, and metadata payload) to be uploaded to Qdrant.
* Points are buffered and uploaded in chunks of size `QDRANT_BATCH_SIZE` to optimize performance.
* The process includes error handling and retry logic to avoid failures halting the entire operation.
* At the end, any remaining points in the buffer are uploaded.
* Summary statistics of processed, failed, and successfully upserted items are printed.



In [13]:
from tqdm import tqdm

print(f"Starting batch embedding and indexing process for {len(df_sample)} movies...")
print(f"Using Gemini Batch Size: {BATCH_SIZE}, Qdrant Upsert Batch Size: {QDRANT_BATCH_SIZE}")

points_to_upsert_buffer = []
total_processed = 0
total_failed_embedding = 0
total_upserted = 0

num_batches = (len(df_sample) + BATCH_SIZE - 1) // BATCH_SIZE

for i in tqdm(range(0, len(df_sample), BATCH_SIZE), total=num_batches, desc="Processing Batches"):
    batch_df = df_sample.iloc[i : i + BATCH_SIZE]
    batch_texts = batch_df['text_for_embedding'].tolist()
    batch_ids = batch_df['id'].tolist()

    if not batch_texts:
        continue

    # Generate embeddings for the current batch of movie texts
    batch_embeddings = get_embeddings_batch(batch_texts, task_type="RETRIEVAL_DOCUMENT")

    # Check if embeddings were successfully generated and correspond to batch size
    if batch_embeddings and len(batch_embeddings) == len(batch_texts):
        for j in range(len(batch_ids)):
            item_id = batch_ids[j]
            item_embedding = batch_embeddings[j]
            row_data = batch_df.iloc[j]

            # Prepare metadata payload for this movie
            payload = create_payload(row_data, payload_columns)

            # Create a Qdrant PointStruct with id, embedding vector, and payload
            point = models.PointStruct(
                id=int(item_id),
                vector=item_embedding.values,
                payload=payload
            )
            points_to_upsert_buffer.append(point)

        total_processed += len(batch_ids)

    else:
        print(f"Failed to get embeddings for batch starting at index {i}. Skipping {len(batch_ids)} items.")
        total_failed_embedding += len(batch_ids)
        continue

    # Upload buffered points to Qdrant if buffer reached batch size or end of data
    if len(points_to_upsert_buffer) >= QDRANT_BATCH_SIZE or (i + BATCH_SIZE >= len(df_sample)):
        if points_to_upsert_buffer:
            try:
                client.upsert(
                    collection_name=COLLECTION_NAME,
                    points=points_to_upsert_buffer,
                    wait=False
                )
                total_upserted += len(points_to_upsert_buffer)
                points_to_upsert_buffer = []
            except Exception as e:
                print(f"Error upserting chunk to Qdrant: {e}")
                points_to_upsert_buffer = []
                time.sleep(5)
                # Pause before retrying to avoid hammering the service after an error

# Upload any remaining points left in buffer after loop completion
if points_to_upsert_buffer:
    print(f"Upserting final remaining chunk of {len(points_to_upsert_buffer)} points.")
    try:
        client.upsert(
            collection_name=COLLECTION_NAME,
            points=points_to_upsert_buffer,
            wait=True
        )
        total_upserted += len(points_to_upsert_buffer)
        points_to_upsert_buffer = []
    except Exception as e:
        print(f"Error upserting final chunk: {e}")

print("Batch embedding and indexing finished.")
print(f"Total items processed (attempted embedding): {total_processed}")
print(f"Total points successfully prepared for upsert: {total_upserted}")


Starting batch embedding and indexing process for 5000 movies...
Using Gemini Batch Size: 100, Qdrant Upsert Batch Size: 768


Processing Batches: 100%|██████████| 50/50 [01:23<00:00,  1.67s/it]

Batch embedding and indexing finished.
Total items processed (attempted embedding): 5000
Total points successfully prepared for upsert: 5000


In [14]:
# Waiting for collection to settle
time.sleep(5)

try:
    count = client.count(collection_name=COLLECTION_NAME, exact=True)
    print(f"\nVerification: Collection '{COLLECTION_NAME}' now contains {count.count} points.") # it should print 5000

except Exception as e:
    print(f"Error verifying collection count: {e}")


Verification: Collection 'tmdb_movies_sample' now contains 5000 points.


### 11. Search and Recommend Similar Movies Using Vector Embeddings


With all movie vectors indexed in Qdrant, you can now perform semantic search. This allows you to take any user query (such as a phrase, movie description, or concept), convert it into an embedding using the same Gemini model, and retrieve the most similar movie vectors from the collection using cosine similarity.

This function demonstrates how to:

* Generate an embedding from your input query using the Gemini API.
* Perform a similarity search using Qdrant’s `search()` method.
* Retrieve the top `k` most similar movie entries, including their metadata and similarity scores.

This is the final step where the vector database functions as a recommendation engine.


In [15]:
def recommend_movies(query_text, top_k=5):
    """
    Finds movies similar to the query_text using the Qdrant index.

    Args:
        query_text (str): The user's query (e.g., movie title, description, theme).
        top_k (int): The maximum number of recommendations to return.

    Returns:
        list: A list of dictionaries, where each dictionary contains the
              payload (movie details) and similarity score of a recommended movie.
              Returns an empty list if query embedding fails or no results found.
    """
    print(f"Searching for recommendations based on: '{query_text}'")
    # Generate embedding for the user query using Gemini
    query_embedding = get_embeddings_batch(query_text, task_type="RETRIEVAL_QUERY")[0].values

    if query_embedding is None:
        print("Error: Could not generate embedding for the query.")
        return []

    try:
        # Perform a semantic search on Qdrant using the query vector
        search_result = client.search(
            collection_name=COLLECTION_NAME,
            query_vector=query_embedding,
            limit=top_k,
            with_payload=True
        )

        recommendations = []
        if search_result:
            print(f"Found {len(search_result)} potential recommendations:")
            for hit in search_result:
                recommendation = {
                    "id": hit.id,
                    "score": hit.score,
                    "payload": hit.payload
                }
                recommendations.append(recommendation)
        else:
            print("No recommendations found matching the query.")

        return recommendations

    except Exception as e:
        print(f"Error during Qdrant search: {e}")
        return []

## Try Out Your Movie Recommender
You can now try querying your movie recommender by describing a theme, genre, or concept in natural language. The system will return the most semantically similar movies from your dataset based on vector similarity search using Gemini-generated embeddings.

In this example, a query like "spy and action based movies" is used to retrieve 5 similar movie entries.

In [16]:
query = "spy and action based movies"
recommendations = recommend_movies(query, top_k=5)

if recommendations:
    print("\n--- Recommendations ---")
    for rec in recommendations:
        print(f"  - Score: {rec['score']:.4f}")
        print(f"    Title: {rec['payload'].get('title', 'N/A')}")
        print(f"    Genre: {rec['payload'].get('genres', 'N/A')}")
        print(f"    Year: {rec['payload'].get('release_year', 'N/A')}")
        print("-" * 10)

Searching for recommendations based on: 'spy and action based movies'
Found 5 potential recommendations:

--- Recommendations ---
  - Score: 0.6662
    Title: The Tiger Gang
    Genre: Drama, Action, Thriller
    Year: 1971.0
----------
  - Score: 0.6588
    Title: From Istanbul with Orders to Kill
    Genre: 
    Year: 1968.0
----------
  - Score: 0.6508
    Title: Jatt James Bond
    Genre: Comedy, Drama, Thriller
    Year: 2014.0
----------
  - Score: 0.6488
    Title: The Blue Panther
    Genre: Action, Crime, Thriller
    Year: 1965.0
----------
  - Score: 0.6454
    Title: Spies, Lies & Naked Thighs
    Genre: Comedy, TV Movie
    Year: 1988.0
----------


<ipython-input-15-008662d28af0>:24: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(
